In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
import scqubits as scq


from qiskit_metal.analyses.quantization.lumped_capacitive import load_q3d_capacitance_matrix
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem
                        
from scipy.constants import speed_of_light as c_light

import matplotlib.pyplot as plt
%matplotlib inline

### 1. load fluxonium cell Q3d simulation results

Loading the Maxwell capacitance matrices for the design as shown in the screenshot below:

<img src="fluxonium_transmon_coupled.png" width=600 height=600 />


where we have a transmon coupled to a fluxonium through a direct coupler. 

For a simple introduction on Maxwell capacitance matrix, check out the following resources:
https://www.fastfieldsolvers.com/Papers/The_Maxwell_Capacitance_Matrix_WP110301_R02.pdf

In [ ]:
path = './Fluxonium_8p5MHz_cmat.txt'
flux_mat, _, _, _ = load_q3d_capacitance_matrix(path)

### load transmon cell Q3d simulation results

In [ ]:
path = './Transmon_5p5GHz_fQ_cmat.txt'
transmon_mat, _, _, _ = load_q3d_capacitance_matrix(path)

### 2. Create LOM cells from capacitance matrices
#### Setting cell objects corresponding to the capacitance simulation results 
`coupler_pad_Q1` and `coupler_pad_Q2` refer to the same node corresponding to the direct coupler between the qubits but are different names in the capacitance matrix results file. In order to merge the two capacitance matrices in the LOM analysis, we need to rename them to be the same name. 

The following three parameters, `ind_dict`, `jj_dict`, `cj_dict`, all have the same structure. Each is a dictionary where the keys are tuples, giving the nodes that a junction is in between, and the values specifying the relevant values associated with the junction. `ind_dict` lets you specify the junction inductance in nH; `jj_dict` specifies the Josephson junction name (you can give the junction any name you wish; just need to be consistent with the name); `cj_dict` specifies the junction capacitance in fF. In the case of the fluxonium, we will set $E_j$ and $E_l$ directly later instead of deriving from the junction inductance; and since we are only concerned with capacitive coupling here (what's currently supported), `ind_dict` can just be a placeholder whose actual value is not important.

In [ ]:
# cell 1

opt1 = dict(
    node_rename = {'coupling_pad_Q1': 'coupling'}, 
    cap_mat = flux_mat,
    ind_dict = {('pad_bot_Q1', 'pad_top_Q1'): 1},  # placeholder inductance here; only used for node-basis transformation and reduction
    jj_dict = {('pad_bot_Q1', 'pad_top_Q1'):'j1'},
)
cell_1 = Cell(opt1)

# cell 2
opt2 = dict(
    node_rename = {'coupling_pad_Q2': 'coupling'},
    cap_mat = transmon_mat,
    ind_dict = {('pad_bot_Q2', 'pad_top_Q2'): 12.31},
    jj_dict = {('pad_bot_Q2', 'pad_top_Q2'):'j2'},
)
cell_2 = Cell(opt2)

### 3. Create subsystems
#### Creating the four subsystems, corresponding to the 2 qubits

`Subsystem` takes three required arguments. The four currently supported system types are `TRANSMON`, `FLUXONIUM`, `TL_RESONATOR` (transmission line resonator) and `LUMPED_RESONATOR`. `nodes` lets you specify which node the subsystem should be mapped to in the cells. They should be consistent with the node names you have given previously. `q_opts` lets specify any optional parameters you want to give. For example, for the fluxonium, you can provide `scqubits` parameters such as `EJ`, `EL` and `flux` here.

In [ ]:
# subsystem 1: fluxonium
fluxonium = Subsystem(name='fluxonium', sys_type='FLUXONIUM', nodes=['j1'], q_opts={'EJ':4860, 'EL':1140, 'flux': .5})


# subsystem 2: transmon
transmon = Subsystem(name='transmon', sys_type='TRANSMON', nodes=['j2'], q_opts={'ncut': 150, 'truncated_dim':10})

### 4. Creat the composite system from the cells and the subsystems

In [ ]:
composite_sys = CompositeSystem(
    subsystems=[fluxonium, transmon], 
    cells=[cell_1, cell_2], 
    grd_node='ground_main_plane')

The `circuitGraph` object encapsulates the lumped model circuit analysis (i.e., LOM analysis) and contain the intermediate as well as final L and C matrices, their inverses needed to construct the Hamiltonian of the composite system. For more details on the meaning and calculation of these matrices, check out https://arxiv.org/pdf/2103.10344.pdf.

Just to note that you can use the analysis without needing to know any detail about this object.

In [ ]:
cg = composite_sys.circuitGraph()
print(cg)

### 5. Generate the hilberspace from the composite system, leveraging the scqubits package

`add_interaction()` adds the interaction terms between the subsystems. Currently, capacitive coupling is supported (which is extracted by from off-diagonal elements in the C matrices, see *eqn 12, 13* in https://arxiv.org/pdf/2103.10344.pdf ) and contribute to the interaction.

In [ ]:
hilbertspace = composite_sys.create_hilbertspace()
hilbertspace = composite_sys.add_interaction()
print(hilbertspace)

### 6. Print the results

Print the calculated Hamiltonian parameters from diagonalized composite system Hamiltonian.

The diagonal elements of the $\chi$ matrix are the anharmonicities of the respective subsystems and the off-diagonal the dispersive shifts between them.

In [ ]:
hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace, evals_count=30)

In [ ]:
hamiltonian_results['chi_in_MHz'].to_dataframe()

The $\chi$'s between the subsystems are based on the coupling strengths, $\it{g}$'s between them (which are computed using the coupling capacitance (currently capacitive coupling is supported) and zero point fluctuations of the subsystem's charge operator at the coupling location)

In [ ]:
composite_sys.compute_gs()

In [ ]:
fluxonium.h_params

In [ ]:
transmon.h_params

### 7. let's sweep some parameters now

Let's sweep the flux from 0 to 1 in a grid of 100 points in unit of flux quantum using `scQubits`'s sweeping library.

In [ ]:
 _sys = hilbertspace.subsys_list[0] # fluxonium

def update_hilbertspace(param_val):
    _sys.flux = param_val
    
param_name = 'flux'
param_vals = np.linspace(0, 1, 101)

sweep = scq.ParameterSweep(
    paramvals_by_name={param_name: param_vals},
    evals_count=30,
    hilbertspace=hilbertspace,
    subsys_update_info={param_name: [_sys]},
    update_hilbertspace=update_hilbertspace,
)

#### Plot transition frequencies as a function of the flux

0->1 transition for the transmon

0->1 transition for the fluxonium

0->2 transition for the fluxonium

In [ ]:
wq_t = sweep.transitions(False, [],(0, 0), (0, 1))[1][0]
wq_f = sweep.transitions(False, [],(0, 0), (1, 0))[1][0]
wq_f_02 = sweep.transitions(False, [],(0, 0), (2, 0))[1][0]


plt.figure(figsize=(15, 6))
plt.plot(param_vals, wq_t, 'ob', label='transmon')
plt.plot(param_vals, wq_f, 'g-', label='fluxonium')
plt.plot(param_vals, wq_f_02, 'r-', label='fluxonium_02')


plt.xticks(param_vals[::10], rotation=45)

plt.xlabel(r'flux ($\Phi_{ext}/\Phi_0$)')
plt.ylabel(r'qubit freq (MHz)')
plt.ylim([5550, 5650])

plt.grid()
plt.legend(fontsize=14)

#### The dispersive shift, $\chi$ between the two qubits as a function of the flux

In [ ]:
wq_f = sweep.transitions(False, [],(0, 0), (1, 0))[1][0]
wq_f_t = sweep.transitions(False, [],(0, 1), (1, 1))[1][0]


chi = wq_f_t - wq_f

plt.figure(figsize=(15, 6))
plt.plot(param_vals, chi, 'ob')

plt.xticks(param_vals[::10], rotation=45)

plt.xlabel(r'flux ($\Phi_{ext}/\Phi_0$)')
plt.ylabel(r'$\chi$ (MHz)')
#plt.ylim([5550, 5650])

plt.grid()
plt.legend(fontsize=14)

#### Zooming on the fluxonium sweet spot: its 0->1 transition as a function of the flux

In [ ]:
wq_f = sweep.transitions(False, [],(0, 0), (1, 0))[1][0]


plt.figure(figsize=(15, 6))
plt.plot(param_vals, wq_f, 'ob')


plt.xticks(param_vals[::10], rotation=45)

plt.xlabel(r'flux ($\Phi_{ext}/\Phi_0$)')
plt.ylabel(r'fluxonium freq (MHz)')

plt.grid()
plt.legend(fontsize=14)

In [ ]:
scq.get_units()
scq.set_units('MHz')

#### Coherences of the fluxonium as a function of the flux

For more information on their calculations and assumptions made, check out `scqubits` documentations:
https://scqubits.readthedocs.io/en/latest/guide/guide-noise.html

In [ ]:
_sys.plot_t1_effective_vs_paramvals(param_name='flux', 
                                    param_vals=param_vals)

In [ ]:
_sys.plot_coherence_vs_paramvals(param_name='flux', param_vals=param_vals)